In [1]:
import numpy as np
from sklearn.datasets import load_iris,make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import classification_report

Generate Dataset

In [2]:
x,y = make_classification(
    n_classes=2,
    n_features=10,
    n_samples=1000,
    n_informative=8,
    n_redundant=2,
    n_repeated=0,
    random_state=42
)

Hyperparameter with GridSearchCV

In [4]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

model = LogisticRegression(max_iter=2000)

params_dict = {
    'C':[1,2,3,4,5],
    'solver':['lbfgs','saga','liblinear','newton-cg','sag']
}


grid_search = GridSearchCV(
    estimator=model,
    param_grid=params_dict,
    cv=3,
    verbose=1,
    scoring='accuracy'
) 

grid_search.fit(x_train,y_train)

print(f'Best Params {grid_search.best_params_}')
print(f'Best Score {grid_search.best_score_}')

best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

print('classification_report')
print(classification_report(y_test,y_pred))

Fitting 3 folds for each of 25 candidates, totalling 75 fits
Best Params {'C': 1, 'solver': 'liblinear'}
Best Score 0.7212084518412135
classification_report
              precision    recall  f1-score   support

           0       0.73      0.62      0.67       106
           1       0.63      0.73      0.68        94

    accuracy                           0.68       200
   macro avg       0.68      0.68      0.67       200
weighted avg       0.68      0.68      0.67       200



Hyperparameter with Optuna

In [5]:
import optuna
from sklearn.model_selection import cross_val_score

/Users/mohitrajnayak/Data Sci Work/Data Sci REPO/Machine-Learning/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def objective(trial):

    C = trial.suggest_float("C",1,5)
    solver = trial.suggest_categorical("solver",['lbfgs','saga','liblinear','newton-cg','sag'])
    
    LogisticRegression(C=C,solver=solver,max_iter=1000)
    score = cross_val_score(model,x,y,cv=3,scoring='accuracy')
    accuracy = np.mean(score)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=10)

print(f'Best Params {study.best_params}')
print(f'Best Score {study.best_value}')

[I 2025-09-09 20:38:58,108] A new study created in memory with name: no-name-36e09b91-2b80-4dcc-bd5e-473a78074a1b
[I 2025-09-09 20:38:58,115] Trial 0 finished with value: 0.6889973806141471 and parameters: {'C': 3.6330059982611265, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.6889973806141471.
[I 2025-09-09 20:38:58,121] Trial 1 finished with value: 0.6889973806141471 and parameters: {'C': 4.629716574388645, 'solver': 'saga'}. Best is trial 0 with value: 0.6889973806141471.
[I 2025-09-09 20:38:58,125] Trial 2 finished with value: 0.6889973806141471 and parameters: {'C': 2.926707378009225, 'solver': 'liblinear'}. Best is trial 0 with value: 0.6889973806141471.
[I 2025-09-09 20:38:58,132] Trial 3 finished with value: 0.6889973806141471 and parameters: {'C': 1.9894865906148524, 'solver': 'liblinear'}. Best is trial 0 with value: 0.6889973806141471.
[I 2025-09-09 20:38:58,136] Trial 4 finished with value: 0.6889973806141471 and parameters: {'C': 3.2079158555774696, 'solver': 'newt

Best Params {'C': 3.6330059982611265, 'solver': 'newton-cg'}
Best Score 0.6889973806141471


In [8]:
best_model = LogisticRegression(**study.best_params,max_iter=1000)
best_model.fit(x_train,y_train)

y_pred = best_model.predict(x_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.73      0.62      0.67       106
           1       0.63      0.73      0.68        94

    accuracy                           0.68       200
   macro avg       0.68      0.68      0.67       200
weighted avg       0.68      0.68      0.67       200

